DISTRICT HEATING AND COOLING NETWORKS IN THE RESIDENTIAL SECTOR

In [ ]:
import os 
import pandas as pd
import geopandas as gpd
from energyemissionsregio.config import DATA_PATH
from energyemissionsregio.disaggregation import distribute_data_equally
from energyemissionsregio.utils import solve_proxy_equation

In [ ]:
final_energy_savings = 599 #ktoe

proxy

In [ ]:
proxies = ["final_energy_consumption_in_machinery_industries",
        "final_energy_consumption_in_food_beverages_and_tobacco_industries",
        "final_energy_consumption_in_transport_equipment_industries",
        "final_energy_consumption_in_construction",
        "final_energy_consumption_in_textile_and_leather_industries",
        "final_energy_consumption_in_households"]

proxy_data_dict = {}

for proxy in proxies:
    proxy_data = pd.read_csv(os.path.join("..", "data", "disaggregated_data", "FEC", f"{proxy}.csv"))
    proxy_data = proxy_data[proxy_data["region_code"].str.startswith("ES")].copy()
    proxy_data["value_confidence_level"] = 5 # VERY HIGH
    proxy_data_dict[proxy] = proxy_data

In [ ]:
var_name = "cproj_annual_mean_temperature_cooling_degree_days"

In [ ]:
proxy_data = pd.read_csv(os.path.join(DATA_PATH, "..", "climate_projections", "ES", var_name, "2020.csv"))
proxy_data = proxy_data[proxy_data["climate_experiment"] == "RCP4.5"].copy()

proxy_data.drop(columns="climate_experiment", inplace=True)


In [ ]:
target_var = "cproj_annual_mean_temperature_cooling_degree_days"

target_data = pd.read_csv(os.path.join(DATA_PATH, "..", "climate_projections", "ES", target_var, "2020.csv"))
target_data = target_data[target_data["climate_experiment"] == "RCP4.5"].copy()

target_data.drop(columns="climate_experiment", inplace=True)

target_data["value_confidence_level"] = 5

In [ ]:
regions_gdf = gpd.read_file(os.path.join(DATA_PATH, "..", "..", "02_processed", "shapefiles", "LAU.shp"))
lau_regions = regions_gdf[regions_gdf["code"].str.startswith("ES")][["code"]].copy()

In [ ]:
proxy_confidence_level = 3

In [ ]:
disagg_data = distribute_data_equally(target_data, "NUTS3", lau_regions, proxy_confidence_level)

In [ ]:
disagg_data

In [ ]:
proxy_data_dict["cooling_degree_days"] = disagg_data

In [ ]:
proxy_data_dict["cooling_degree_days"] 

In [ ]:
proxy_equation = """final_energy_consumption_in_machinery_industries +  final_energy_consumption_in_food_beverages_and_tobacco_industries + final_energy_consumption_in_transport_equipment_industries + final_energy_consumption_in_construction + final_energy_consumption_in_textile_and_leather_industries + (final_energy_consumption_in_households * cooling_degree_days)"""

proxy_df = solve_proxy_equation(proxy_equation, proxy_data_dict)

In [ ]:
proxy_df

binary disaggregation criteria

In [ ]:
population = pd.read_csv(os.path.join(DATA_PATH, "population.csv"))
population = population[population["region_code"].str.startswith("ES")]

In [ ]:
population = population[["region_code", "value"]].copy()
population.rename(columns={"value": "population"}, inplace=True)

In [ ]:
proxy_df = pd.merge(proxy_df, population, on = "region_code")

In [ ]:
proxy_df['value'] = proxy_df.apply(lambda row: row['value'] if row['population'] > 45000 else 0, axis=1) 

In [ ]:
proxy_df[proxy_df["value"] > 0]

Disaggregation

In [ ]:
proxy_df["share"] = proxy_df["value"] / proxy_df["value"].sum()

proxy_df["value"] = proxy_df["share"] * final_energy_savings

In [ ]:
final_df = proxy_df[["region_code", "value"]].copy()

In [ ]:
final_df.to_csv(os.path.join("..", "data", "NECP_disaggregation", "disaggregated_data",  "FES_M2.10_district_heating_and_cooling.csv"), index=False)